# Run only once to install the software

In [8]:
!pip install -U transformers
!pip install -U spacy
!pip install -U ludwig
!pip install -U tensorflow==2.10 

!python -m spacy download en_core_web_sm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached transformers-4.27.4-py3-none-any.whl (6.8 MB)
  Attempting uninstall: transformers
    Found existing installation: transformers 4.21.3
    Uninstalling transformers-4.21.3:
      Successfully uninstalled transformers-4.21.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ludwig 0.7.4 requires transformers<4.22,>=4.10.1, but you have transformers 4.27.4 which is incompatible.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached transformers-4.21.3-py3-none-any.whl (4.7 MB)
  Attempting uninstall: transformers
    Found existing installation: transformers 4.27.4
    Uninstalling transformers-4

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 578.1/578.1 MB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 72.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 KB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.7/438.7 KB 29.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 64.3 MB/s eta 0:00:00
  Attempting uninstall: keras
    Found existing installation: keras 2.11.0
    Uninstalling keras-2.11.0:
      Successfully uninstalled keras-2.11.0
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.11.0
    Uninstalling tensorflow-estimator-2.11.0:
      Successfully uninstalled tensorflow-estimator-2.11.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.11.2
    Uninstalling tensorboard-2.11.2:
      Successf

/usr/local/lib/python3.9/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
2023-03-30 12:34:46.005251: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-03-30 12:34:47.090687: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-03-30 12:34:47.090861: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-03-30 12:34:47.090891: W tensorflow/compiler/tf2tensorrt

# Import models

In [9]:
from transformers import pipeline
import json
import spacy
from spacy import displacy
from ludwig.api import LudwigModel
from ludwig.datasets import agnews
import logging
nlp = spacy.load("en_core_web_sm")

In [3]:
def getFactText():
    allFactTexts={"parts":[], "fullText":[]}
    with open('cases-9.json', encoding='utf-8') as json_file:
        data = json.load(json_file)
        for case in data:
            caseFullText=""
            for contentItem in case["content"]:
                for textBulk in case["content"][contentItem]:
                    if textBulk["content"]=="THE FACTS":
                        for subText in textBulk["elements"]:
                            if subText["content"]=="I.\xa0\xa0THE CIRCUMSTANCES OF THE CASE":
                                for actualText in subText["elements"]:
                                    allFactTexts["parts"].append(actualText["content"].replace("\xa0\xa0", " "))
                                    caseFullText=caseFullText + "\n " + actualText["content"].replace("\xa0\xa0", " ")
            allFactTexts["fullText"].append(caseFullText)
    return allFactTexts

In [5]:
facts = getFactText()

factToTestOn=facts["parts"][13]
longText=facts["fullText"][3]
print(factToTestOn)
print(longText)

6. According to the applicant, on 19 July 2010 he arrived in Moscow from  with a view to finding a seasonal job and marrying. He soon learnt that his brothers had been arrested and charged with participation in an extremist religious organisation. He also learnt that he was suspected by the Uzbekistani authorities of having founded an extremist religious organisation himself.

 5. The applicant was born in 1974 and lives in .
 6. According to the applicant, on 19 July 2010 he arrived in Moscow from  with a view to finding a seasonal job and marrying. He soon learnt that his brothers had been arrested and charged with participation in an extremist religious organisation. He also learnt that he was suspected by the Uzbekistani authorities of having founded an extremist religious organisation himself.
 A. Application for asylum and refugee status
 B. Arrest and detention pending extradition
 C. Arrest and detention with view to expulsion


# Question / answering

In [10]:
question_answerer = pipeline("question-answering", model='distilbert-base-uncased-distilled-squad')

context = factToTestOn

In [11]:
result = question_answerer(question="When did the applicant arrive in Moscow?",     context=context)

In [12]:
print(result)

{'score': 0.9752744436264038, 'start': 34, 'end': 46, 'answer': '19\xa0July 2010'}


# Entity highlighting

In [13]:

doc = nlp(factToTestOn)
displacy.render(doc, style="ent")

'<div class="entities" style="line-height: 2.5; direction: ltr">\n<mark class="entity" style="background: #e4e7d2; padding: 0.45em 0.6em; margin: 0 0.25em; line-height: 1; border-radius: 0.35em;">\n    6\n    <span style="font-size: 0.8em; font-weight: bold; line-height: 1; border-radius: 0.35em; vertical-align: middle; margin-left: 0.5rem">CARDINAL</span>\n</mark>\n. According to the applicant, on \n<mark class="entity" style="background: #bfe1d9; padding: 0.45em 0.6em; margin: 0 0.25em; line-height: 1; border-radius: 0.35em;">\n    19\xa0July 2010\n    <span style="font-size: 0.8em; font-weight: bold; line-height: 1; border-radius: 0.35em; vertical-align: middle; margin-left: 0.5rem">DATE</span>\n</mark>\n he arrived in \n<mark class="entity" style="background: #feca74; padding: 0.45em 0.6em; margin: 0 0.25em; line-height: 1; border-radius: 0.35em;">\n    Moscow\n    <span style="font-size: 0.8em; font-weight: bold; line-height: 1; border-radius: 0.35em; vertical-align: middle; margi

# Summarisation

In [14]:
summarizer = pipeline("summarization", model="facebook/bart-base")
print(summarizer(longText, max_length=40, min_length=30, do_sample=False))


[{'summary_text': '1. Application for asylum and refugee status 5. The applicant was born in 1974 and lives in . 6. According to the applicant, on 19\xa0July 2010 he arrived in Moscow from'}]


# Supervised Machine Learning

In [16]:
train_df = agnews.load()


config = {
  "input_features": [
    {
      "name": "title",            # The name of the input column
      "type": "text",             # Data type of the input column
      "encoder": {
            "type": "parallel_cnn"
       }                          # The model architecture we should use for encoding this column
    }
  ],
  "output_features": [
    {
      "name": "class",
      "type": "category",
    }
  ]
}

model = LudwigModel(config, logging_level=logging.INFO)


In [ ]:
train_stats, preprocessed_data, output_directory = model.train(dataset=train_df)

INFO:ludwig.utils.print_utils:
INFO:ludwig.utils.print_utils:╒════════════════════════╕
INFO:ludwig.utils.print_utils:│ EXPERIMENT DESCRIPTION │
INFO:ludwig.utils.print_utils:╘════════════════════════╛
INFO:ludwig.utils.print_utils:
INFO:ludwig.api:╒══════════════════╤═════════════════════════════════════════════════════════════════════════════════════════╕
│ Experiment name  │ api_experiment                                                                          │
├──────────────────┼─────────────────────────────────────────────────────────────────────────────────────────┤
│ Model name       │ run                                                                                     │
├──────────────────┼─────────────────────────────────────────────────────────────────────────────────────────┤
│ Output directory │ /content/results/api_experiment_run                                                     │
├──────────────────┼─────────────────────────────────────────────────────────────────

Training:   0%|          | 163/2233000 [00:25<82:53:30,  7.48it/s]

In [ ]:
test_stats, predictions, output_directory = model.evaluate(
  test_df,
  collect_predictions=True,
  collect_overall_stats=True
)

In [ ]:
from ludwig.visualize import confusion_matrix

confusion_matrix(
  [test_stats],
  model.training_set_metadata,
  'class',
  top_n_classes=[5],
  model_names=[''],
  normalize=True,
)



In [ ]:
text_to_predict = pd.DataFrame({
  "title": [
    "Google may spur cloud cybersecurity M&A with $5.4B Mandiant buy",
    "Europe struggles to meet mounting needs of Ukraine's fleeing millions",
    "How the pandemic housing market spurred buyer's remorse across America",
  ]
})

predictions, output_directory = model.predict(text_to_predict)